In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns; sns.set(style="ticks", color_codes=True)
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import seaborn as sns
import cufflinks as cf
import plotly.express as px
%matplotlib inline

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected = True)
cf.go_offline() 

In [ ]:
df= pd.read_csv("/kaggle/input/health-care-data-set-on-heart-attack-possibility/heart.csv")


In [ ]:
df.head(5)

In [ ]:
df.isnull().sum()

In [ ]:
fig = px.histogram(df['age'], nbins=11, labels={'value': "Age"} , 
                   title ='5000 Dice RollHistogram',
                   marginal = 'violin',
                   color_discrete_sequence=['blue']
                  )

fig.update_layout(
xaxis_title_text= 'Age',
yaxis_title_text = 'Count of People',
bargap=0.2, showlegend= False)
fig

In [ ]:

dict1 ={ 1 : 'Male' , 0 : 'Female'}

df['gender'] = df['sex'].map(dict1)
    
df.head()

In [ ]:

px.box(df, x= 'gender' , y='target')

In [ ]:
px.bar(df, x= 'age' , y='target', color='gender' , title= 'heart attack patoents age range and sex',
      labels = { 'output': 'Number of patients', 'Age': 'Age od patient'})

In [ ]:
fig = plt.figure(figsize=(20,10))
sns.heatmap(df.corr(), annot= True, fmt = '.2f', cmap = 'Blues')

In [ ]:
df_new=df.drop(columns= ['trestbps', 'chol' , 'fbs' , 'restecg' , 'gender'])
df_new.head()

In [ ]:

Features=sns.PairGrid(df_new, hue='target')

Features.map_diag(plt.hist)
Features.map_offdiag(plt.scatter)

In [ ]:
X = df_new.drop(columns= ['target'])
y= df_new['target']

# Forecasting

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.metrics import *
from sklearn import metrics
from sklearn.metrics import classification_report

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [ ]:
lgr_pipline  = Pipeline([("scaler", StandardScaler()), ("LogisticRegression", LogisticRegression())])
rfc_pipline = Pipeline([("scaler", StandardScaler()), ("RandomForestClassifier", RandomForestClassifier())])
knn_pipline = Pipeline([("scaler", StandardScaler()), ("KNeighborsClassifier", KNeighborsClassifier())])
gnb_pipline = Pipeline([("scaler", StandardScaler()), ("GaussianNB", GaussianNB())])
sgd_pipline = Pipeline([("scaler", StandardScaler()), ("SGDClassifier", SGDClassifier())])
dt_pipline = Pipeline([("scaler", StandardScaler()), ("DecisionTreeClassifier", DecisionTreeClassifier())])


In [ ]:
pip_dict1 ={0:'Logistic Regression' , 1:'RandomForestClassifier' , 2: 'KNeighborsClassifier' ,
            3: 'GaussianNB', 4: 'SGDClassifier', 5: 'DecisionTreeClassifier' }

In [ ]:
piplines1=[lgr_pipline, rfc_pipline , knn_pipline, gnb_pipline  , sgd_pipline , dt_pipline ]

In [ ]:
scores_df = pd.DataFrame(columns = ["Model", "CVScores"])
for i, pipe in enumerate(piplines1):
    score = cross_val_score(pipe, X, y, cv = 10)
    print(pip_dict1[i], ": ", score.mean())

In [ ]:
grid_params = [
    {"classifier": [RandomForestClassifier()],
    "classifier__n_estimators": [50,100,150,200,250,300],
    "classifier__criterion": ["gini", "entropy"],
    "classifier__max_features": ["auto", "sqrt", "log2"],
    },
    
    {"classifier": [KNeighborsClassifier()],
     "classifier__n_neighbors": [2,3,4,5,6],
     "classifier__algorithm": ['auto', 'ball_tree', 'kd_tree', 'brute'],
     "classifier__leaf_size": [10,20,30,40,50],
    
    },
    {"classifier": [LogisticRegression()],
     "classifier__penalty": ['l1', 'l2', 'elasticnet'],
     "classifier__fit_intercept": [True, False],
     "classifier__solver":["newton-cg", "lbfgs", "liblinear", "sag", "saga"],
     "classifier__max_iter": [50, 100, 500, 1000, 2500]
    }
    
    
]

In [ ]:

pipeline_new = Pipeline([("scaler", StandardScaler()), ("classifier", RandomForestClassifier())])

random_search = RandomizedSearchCV(estimator = pipeline_new, param_distributions = grid_params, scoring = 'neg_mean_absolute_error', n_jobs= -1, cv = 8, verbose = 10, random_state = 42)


In [ ]:
best_model = random_search.fit(X_train, y_train)

In [ ]:
best_model.best_params_

In [ ]:
pipeline_rf = Pipeline([('scaler', StandardScaler()), 
                           ('RandomForest', RandomForestClassifier(max_features='sqrt', n_estimators=200))])

In [ ]:
model = pipeline_rf.fit(X_train, y_train)

In [ ]:
rf_train_predict = pd.DataFrame({'actual' : y_train,
                                 'predicted' : model.predict(X_train)})
rf_train_predict.head()

In [ ]:
rf_test_predict = pd.DataFrame({'actual' : y_test,
                                 'predicted' : model.predict(X_test)})
rf_test_predict.head()

In [ ]:
print('Accuracy Score for train dataset : ' , metrics.accuracy_score(rf_train_predict.actual, rf_train_predict.predicted))
print('Accuracy Score for test dataset : ' , metrics.accuracy_score(rf_test_predict.actual, rf_test_predict.predicted))


In [ ]:
conn_cm_test = metrics.confusion_matrix(rf_test_predict.actual, rf_test_predict.predicted, [1,0])
sns.heatmap(conn_cm_test, fmt= '.2f', annot=True)

In [ ]:
print(metrics.classification_report(rf_test_predict.actual, rf_test_predict.predicted))